In [ ]:
import pandas as pd
file_name='data/FT_Camp_5/Train.csv'
l_data = pd.read_csv(file_name,nrows=5)

In [ ]:
data = pd.read_csv(file_name)
data.set_index('stockcode',inplace=True)
data.shape

In [ ]:
#distribution
import matplotlib.pyplot as plt
import seaborn as sns
fig,ax=plt.subplots(1,2,figsize=(14,7))
sns.countplot(x='fake',data=data,ax=ax[0])
data['fake'].value_counts().plot(x=None,y=None,kind='pie',ax=ax[1],autopct='%1.2f%%')
plt.show()

In [ ]:
Xfraud=data.loc[data['fake']==1]
Xnonfraud=data.loc[data['fake']==0]

In [ ]:
import numpy as np
correlationNonfraud=Xnonfraud.loc[:,data.columns!='fake'].corr()
mask=np.zeros_like(correlationNonfraud)
indices = np.triu_indices_from(correlationNonfraud)
mask[indices]=True
grid_kws={'width_ratios':(.9,.9,.05),'wspace':0.2}
f,(ax1,ax2,cbar_ax)=plt.subplots(1,3,gridspec_kw=grid_kws,figsize=(20,10))
cmap=sns.diverging_palette(220,8,as_cmap=True)
ax1=sns.heatmap(correlationNonfraud,ax=ax1,vmin=-1,vmax=1,cmap=cmap,square=False,linewidths=0.5,mask=mask,cbar=False)
ax1.set_xticklabels(ax1.get_xticklabels(),size=16)
ax1.set_yticklabels(ax1.get_yticklabels(),size=16)
ax1.set_title('Normal',size=20)

correlationfraud=Xfraud.loc[:,data.columns!='fake'].corr()
# mask=np.zeros_like(correlationNonfraud)
# indices = np.triu_indices_from(correlationNonfraud)
# mask[indices]=True
# grid_kws={'width_ratios':(.9,.9.,.05),'wspace':0.2}
# f,(ax1,ax2,cbar_ax)=plt.subplots(1,3,gridspec_kw=grid_kws,figsize=(14,9))
# cmap=sns.diverging_palette(220,8,as_cmap=True)
ax2=sns.heatmap(correlationfraud,ax=ax2,vmin=-1,vmax=1,cmap=cmap,square=False,linewidths=0.5,mask=mask,yticklabels=False,cbar_ax=cbar_ax,cbar_kws={'orientation':'vertical','ticks':[-1,-0.5,0,0.5,1]})
ax2.set_xticklabels(ax2.get_xticklabels(),size=16)
# ax1.set_yticklabels(ax1,get_yticklabels(),size=16)
ax2.set_title('Fraud',size=20)
cbar_ax.set_yticklabels(cbar_ax.get_yticklabels(),size=14)

In [ ]:
#relationship between opinion and fraud
f,(ax1,ax2)=plt.subplots(2,1,sharex=True,figsize=(16,4))
bins=30
ax1.hist(data['Opinion'][data.fake==1],bins=bins)
ax1.set_title("Fraud")
ax2.hist(data['Opinion'][data.fake==0],bins=bins)
ax2.set_title("Normal")
plt.xlabel('Opinion')
plt.ylabel('Numbers')
plt.yscale('log')
plt.show()
# sns.factorplot(x='Opinion',data=data,kind='count',palette='ocean',size=5,aspect=3)

In [ ]:
data.groupby('fake').size()

In [ ]:
from matplotlib import gridspec as gridspec
v_feat=data.iloc[:,0:35].columns
plt.figure(figsize=(16,28*4))
gs=gridspec.GridSpec(35,1)
for i,cn in enumerate(data[v_feat]):
#     print(i,cn)
    ax=plt.subplot(gs[i])
    sns.distplot(data[cn][data.fake==1],bins=50)
    sns.distplot(data[cn][data.fake==0],bins=100)
    ax.set_xlabel('')
    ax.set_title(str(cn))

In [ ]:
stay_col=['CA_ratio','Quick_ratio','TA_TO_ratio','Curr_ratio','OtherRec_to_Cur','Pre_to_Cur','Neg_Dednp_times','Monetary_to_Cur','CIP_ratio','fake']
data_new=data[stay_col]
data_new.head()

In [ ]:
import warnings
warnings.filterwarnings('ignore')
# 进行特征放缩
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
sc = StandardScaler()
# data_new.head().T
x_feature=stay_col
x_feature.remove('fake')
data_new[x_feature]=sc.fit_transform(data_new[x_feature])
x_val=data_new[x_feature]
y_val=data_new['fake']
names=data_new[x_feature].columns
clf=RandomForestClassifier(n_estimators=10,random_state=123)
clf.fit(x_val,y_val)
# names, clf.feature_importances_
for feature in zip(names,clf.feature_importances_):
    print(feature)

In [ ]:
#未经过特征放缩
from sklearn.ensemble import RandomForestClassifier
x_feature=stay_col
x_feature.remove('fake')
x_val=data_new[x_feature]
y_val=data_new['fake']
names=data_new[x_feature].columns
clf=RandomForestClassifier(n_estimators=10,random_state=123)
clf.fit(x_val,y_val)
# names, clf.feature_importances_
for feature in zip(names,clf.feature_importances_):
    print(feature)

In [ ]:
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize']=(12,6)
importance = clf.feature_importances_
feat_names =names
indices=np.argsort(importance)[::-1]#dao xu
fig=plt.figure(figsize=(20,6))
plt.title('Feature importances by RandomForestClassifier')
plt.bar(range(len(indices)),importance[indices],color='lightblue',align='center')
plt.step(range(len(indices)),np.cumsum(importance[indices]),where='mid',label='CUmulative')
plt.xticks(range(len(indices)),feat_names[indices],rotation='vertical',fontsize=14)
plt.xlim([-1,len(indices)])

In [ ]:
X=data_new[x_feature]
y=data_new['fake']

n_sample =y.shape[0]
n_pos_sample = y[y==0].shape[0]
n_neg_sample = y[y==1].shape[0]
print('样本个数：{}：正样本占{:.2%}：负样本占{:.2%}'.format(n_sample,n_pos_sample/n_sample,n_neg_sample/n_sample))
print('特征维数:',X.shape[1])

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X,y=sm.fit_sample(X,y)
print('经过处理后：')
n_sample = y.shape[0]
n_pos_sample = y[y==0].shape[0]
n_neg_sample = y[y==1].shape[0]
print('样本个数：{}：正样本占{:.2%}：负样本占{:.2%}'.format(n_sample,n_pos_sample/n_sample,n_neg_sample/n_sample))
print('特征维数:',X.shape[1])


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
lr = LogisticRegression(solver='liblinear',verbose=1)
lr.fit(X,y)

In [ ]:
predicted1 = lr.predict(X)
print('Test set accuracy score:{:.5f}'.format(accuracy_score(predicted1,y,)))
data_new[x_feature].head()

In [ ]:
file_test='data/FT_Camp_5/X_test.csv'
test_data=pd.read_csv(file_test)


In [ ]:
X_test=test_data[x_feature]
X_test[x_feature]=sc.fit_transform(X_test[x_feature])
X_test.head()

In [ ]:
X_test['fake']=lr.predict(X_test)
X_test['stockcode']=test_data['stockcode']

In [ ]:
X_test[['stockcode','fake']].to_csv('simple_lr.csv',sep=',',header=True,index=False,encoding='UTF-8')

In [ ]:
data.set_index('stockcode')

In [ ]:
# 该方案使用SMOTE来进行up sampling，使得原先不平衡数据样本个数：7653：正样本占98.58%：负样本占1.42%变成样本个数：15088：正样本占50.00%：负样本占50.00%，经过特征筛选
# 具体为将特征与类别分布不一致的特征筛选出来
# 选出['CA_ratio','Quick_ratio','TA_TO_ratio','Curr_ratio','OtherRec_to_Cur','Pre_to_Cur', 'Neg_Dednp_times','Monetary_to_Cur','CIP_ratio']
# 流动资产比率，速动比率，总资产周转率，流动比率，其他应收款项占流动资产比例，预付款项占流动资产比例，最近三年扣非净利润为负的次数（包括今年），货币资金占流动资产比例，在建工程比率
x_feature

In [ ]:
data_new.to_csv('Stan_fliter_data.csv',index=False)

In [ ]:
data[stay_col].to_csv('fliter_data.csv',index=True)

In [ ]:
stay_col.append('fake')
data[stay_col].head()